# Evaluation of model performance

## Preamble

In [ ]:
import pandas as pd

In [ ]:
import scipy
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from iminuit import cost, Minuit

In [ ]:
from iminuit.cost import LeastSquares

In [ ]:
import hist

In [ ]:
from plotting import watermark

In [ ]:
plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

## $E_\nu$ model

### Load and prepare data

In [ ]:
model_name = "CNN_3d_grandjorasses_nu_energy"
epochs = 75

In [ ]:
df = pd.read_csv(f"{model_name}_n75842_e{epochs}.csv")

In [ ]:
df["total_energy_test"] = df.nu_energy_test
df["total_energy_pred"] = df.nu_energy_pred

In [ ]:
df["corrected_total_energy"] = df.total_energy_pred

### Correct energy scale

In [ ]:
scale2 = df.total_energy_test.mean() / df.corrected_total_energy.mean()

In [ ]:
scale = np.exp(
    (np.log(df.total_energy_test) - np.log(df.corrected_total_energy)).mean()
)

In [ ]:
shift = ((scale * df.corrected_total_energy) - df.total_energy_test).mean()

In [ ]:
shift2 = ((scale2 * df.corrected_total_energy) - df.total_energy_test).mean()

In [ ]:
df["corrected_total_energy2"] = (scale2 * df.corrected_total_energy) - shift2

In [ ]:
df.corrected_total_energy = (scale * df.corrected_total_energy) - shift

In [ ]:
print(scale, scale2, shift, shift2)

Offset should be consistent with zero, small scale factor seems to be needed, consistent with small bias in $\log{E}$

In [ ]:
df["d_corrected_energy"] = df.corrected_total_energy - df.total_energy_test

In [ ]:
bins = np.linspace(0, 5000, 51)
plt.hist(df.total_energy_test, bins=bins, label="true", histtype="step")
plt.hist(df.total_energy_pred, bins=bins, label="uncorrected", alpha=0.5)
plt.hist(df.corrected_total_energy, bins=bins, label="corrected", alpha=0.5)
plt.hist(
    df.corrected_total_energy2, bins=bins, label="corrected - alternative", alpha=0.5
)
plt.xlabel(r" $E\;[\mathrm{GeV}]$")
watermark()
plt.legend()
plt.savefig(f"plots/energy_correction_{model_name}.png")
plt.savefig(f"plots/energy_correction_{model_name}.pdf")

In [ ]:
(df.d_corrected_energy / df.total_energy_test).hist(bins=100, log=True)

In [ ]:
df.d_corrected_energy.hist(bins=100, log=True)

In [ ]:
df.d_corrected_energy[(1000 < df.total_energy_test) & df.total_energy_test < 2000].hist(
    bins=100, log=True
)

### Fit energy resolution

In [ ]:
bins_E_reco = 14

In [ ]:
(2000 - 200) / 15

In [ ]:
df.total_energy_test.min()

In [ ]:
h_dE_rel_test_vs_E_rel_pred = (
    hist.Hist.new.Regular(200, -1000, 1000, name=r"dE")
    .Regular(
        bins_E_reco, 320, 2000, name=r"E_true"
    )  # , transform=hist.axis.transform.log)
    .Double()
)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.fill(df.d_corrected_energy, df.total_energy_test)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.plot()
plt.xlabel(r" $\Delta E\;[\mathrm{GeV}]$")
plt.ylabel(r"true $E\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
# plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.pdf")
# plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.png")

In [ ]:
def model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, loc=mu, scale=sigma)

In [ ]:
mus = []
sigmas = []
bins = []

for bin in range(bins_E_reco):
    h = h_dE_rel_test_vs_E_rel_pred[:, bin]
    entries, edges = h.to_numpy()
    n_bins = len(entries)
    average = np.average(edges[:-1], weights=entries)
    variance = np.average((edges[:-1] - average) ** 2, weights=entries)
    m = Minuit(cost.BinnedNLL(entries, edges, model), average, np.sqrt(variance))
    res = m.migrad()
    if res.valid:
        bins.append(bin)
        mus.append(res.params[0])
        sigmas.append(res.params[1])
    else:
        print(res)

In [ ]:
bin_edges = h_dE_rel_test_vs_E_rel_pred[0, :].to_numpy()[1]
bin_centres = (bin_edges[1:] + bin_edges[:-1]) / 2
bin_half_widths = (bin_edges[1:] - bin_edges[:-1]) / 2

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [mu.value for mu in mus] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[mu.error for mu in mus] / bin_centres[bins],
    linestyle="",
    label=r"$\left<\Delta E\right>$",
    color=colors[0],
)
plt.hlines(0, *plt.xlim(), color="red")
plt.ylabel(r"$\frac{\left<\Delta E\right>}{E_\mathrm{true}}$")
plt.xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/energy_bias.pdf")
plt.savefig("plots/energy_bias.png")

In [ ]:
import sympy

In [ ]:
A, b, E = sympy.symbols("A b E")

In [ ]:
f = A + b / sympy.sqrt(E)

In [ ]:
f_lambda = sympy.lambdify((A, b, E), f)

In [ ]:
def E_model(E, A, b):
    return f_lambda(A, b, E)

In [ ]:
sigma_E_over_E = np.array([sigma.value for sigma in sigmas]) / bin_centres[bins]
d_sigma_E_over_E = (
    [sigma.value for sigma in sigmas]
    / bin_centres[bins]
    * np.sqrt(
        (
            np.array([sigma.error for sigma in sigmas])
            / np.array([sigma.value for sigma in sigmas])
        )
        ** 2
        + (bin_half_widths[bins] / bin_centres[bins]) ** 2
    )
)

In [ ]:
least_squares = LeastSquares(
    bin_centres[bins], sigma_E_over_E, d_sigma_E_over_E, E_model
)

In [ ]:
m = Minuit(least_squares, A=0.1, b=1)  # starting values for α and β

m.migrad()  # finds minimum of least_squares function
res = m.hesse()  # accurately computes uncertainties

In [ ]:
res

In [ ]:
f_pretty = sympy.latex(
    f.subs(
        [
            (A, sympy.Float(res.params[0].value, 1)),
            (b, sympy.Float(res.params[1].value, 2)),
        ]
    )
)

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [sigma.value for sigma in sigmas] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=d_sigma_E_over_E,
    linestyle="",
    label=r"$\sigma\left(\Delta E\right)$",
    color=colors[1],
    fmt="o",
    capsize=3,
)
plt.plot(
    bin_centres[bins],
    E_model(bin_centres[bins], res.params[0].value, res.params[1].value),
)
plt.ylabel(r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{true}}$")
plt.xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")
ax = plt.gca()
watermark()
plt.text(0.6, 0.7, rf"${f_pretty}$", fontsize=14, transform=ax.transAxes)
plt.text(
    0.6,
    0.6,
    rf"$A = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$",
    fontsize=14,
    transform=ax.transAxes,
)
plt.text(
    0.6,
    0.5,
    rf"$b = {res.params[1].value:.1f} \pm {res.params[1].error:.1f}$",
    fontsize=14,
    transform=ax.transAxes,
)

plt.savefig(f"plots/energy_resolution_{model_name}.pdf")
plt.savefig(f"plots/energy_resolution_{model_name}.png")

In [ ]:
import uproot

In [ ]:
filename_test = "root://eospublic.cern.ch//eos/experiment/sndlhc/users/olantwin/advsnd/2024/09/nu12/Default/dataframe_CC_test.root:df"

In [ ]:
events_test = uproot.open(filename_test)

In [ ]:
events_test.keys()

In [ ]:
df_truth = events_test.arrays(
    ["start_x", "start_y", "start_z", "energy_dep_mufilter", "energy_dep_target"],
    library="pd",
)

In [ ]:
df = pd.concat([df, df_truth], axis=1)

In [ ]:
df.pop("Unnamed: 0")

In [ ]:
df["visible"] = (df.energy_dep_mufilter + df.energy_dep_target) / df.nu_energy_test

In [ ]:
from matplotlib.patches import Rectangle, Ellipse

In [ ]:
plt.hist2d(df.start_z, df.start_y)
ax = plt.gca()
ax.add_patch(
    Rectangle((-232, -10), 150, 20, linewidth=2, edgecolor="r", facecolor="none")
)
watermark()
plt.xlabel(r"$z$")
plt.ylabel(r"$y$")


plt.savefig("plots/bins_zy.pdf")
plt.savefig("plots/bins_zy.png")

In [ ]:
plt.hist2d(df.start_x, df.start_y)
ax = plt.gca()
ax.add_patch(Rectangle((0, -10), 20, 20, linewidth=2, edgecolor="r", facecolor="none"))
watermark()
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")

plt.savefig("plots/bins_xy.pdf")
plt.savefig("plots/bins_xy.png")

In [ ]:
h_dE_rel_test_vs_E_rel_pred_pos = (
    hist.Hist.new.Regular(200, -1000, 1000, name=r"dE")
    .Regular(
        bins_E_reco, 320, 2000, name=r"E_true"
    )  # , transform=hist.axis.transform.log)
    .Regular(10, 0, 20, name="x")
    .Regular(10, -10, 10, name="y")
    .Regular(10, -232, -82, name="z")
    .Regular(10, df.visible.min(), df.visible.max(), name="containment")
    .Double()
)

In [ ]:
h_dE_rel_test_vs_E_rel_pred_pos.fill(
    df.d_corrected_energy,
    df.total_energy_test,
    df.start_x,
    df.start_y,
    df.start_z,
    df.visible,
)

In [ ]:
import itertools

In [ ]:
def resolution(h, axis, only=False):
    mus = []
    sigmas = []
    bins = []

    for bin_E, bin_pos in itertools.product(range(bins_E_reco), range(10)):
        bin = [bin_E, hist.sum, hist.sum, hist.sum, hist.sum]
        bin[axis + 1] = bin_pos
        h = h_dE_rel_test_vs_E_rel_pred_pos[:, bin[0], bin[1], bin[2], bin[3], bin[4]]
        entries, edges = h.to_numpy()
        len(entries)
        average = np.average(edges[:-1], weights=entries)
        variance = np.average((edges[:-1] - average) ** 2, weights=entries)
        m = Minuit(cost.BinnedNLL(entries, edges, model), average, np.sqrt(variance))
        res = m.migrad()
        if res.valid:
            bins.append(bin)
            mus.append(res.params[0])
            sigmas.append(res.params[1])
        else:
            print(res)

    As = []
    bs = []
    for i in range(10):
        bin = 4 * [
            hist.sum,
        ]
        bin[axis] = i
        bin_edges = h_dE_rel_test_vs_E_rel_pred_pos[
            0, :, bin[0], bin[1], bin[2], bin[3]
        ].to_numpy()[1]
        bin_centres = (bin_edges[1:] + bin_edges[:-1]) / 2
        bin_half_widths = (bin_edges[1:] - bin_edges[:-1]) / 2
        _bins = list(zip(*bins[::10]))[0]
        _sigmas = sigmas[i::10]
        _sigma_E_over_E = np.array([sigma.value for sigma in _sigmas]) / bin_centres
        _d_sigma_E_over_E = (
            [sigma.value for sigma in _sigmas]
            / bin_centres
            * np.sqrt(
                (
                    np.array([sigma.error for sigma in _sigmas])
                    / np.array([sigma.value for sigma in _sigmas])
                )
                ** 2
                + (bin_half_widths / bin_centres) ** 2
            )
        )
        _least_squares = LeastSquares(
            bin_centres, _sigma_E_over_E, _d_sigma_E_over_E, E_model
        )
        _m = Minuit(_least_squares, A=0.1, b=1)  # starting values for α and β
        _m.migrad()  # finds minimum of least_squares function
        _res = _m.hesse()  # accurately computes uncertainties
        if not only or i == only:
            plt.errorbar(
                bin_centres,
                _sigma_E_over_E,
                xerr=bin_half_widths,
                yerr=_d_sigma_E_over_E,
                linestyle="",
                label=r"$\sigma\left(\Delta E\right)$",
                # color=colors[1],
                fmt="o",
                capsize=3,
            )
            plt.plot(
                bin_centres,
                E_model(bin_centres, _res.params[0].value, _res.params[1].value),
            )
        As.append(_res.params[0])
        bs.append(_res.params[1])
    watermark()
    suffix = ["x", "y", "z", "containment"][axis]
    plt.ylabel(r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{true}}$")
    plt.xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")
    plt.savefig(f"plots/energy_resolution_{suffix}_{model_name}.pdf")
    plt.savefig(f"plots/energy_resolution_{suffix}_{model_name}.png")
    return As, bs

In [ ]:
As_x, bs_x = resolution(h_dE_rel_test_vs_E_rel_pred_pos, axis=0)

In [ ]:
As_y, bs_y = resolution(h_dE_rel_test_vs_E_rel_pred_pos, axis=1)

In [ ]:
As_z, bs_z = resolution(h_dE_rel_test_vs_E_rel_pred_pos, axis=2)

In [ ]:
As_containment, bs_containment = resolution(h_dE_rel_test_vs_E_rel_pred_pos, axis=3)

In [ ]:
As_containment, bs_containment = resolution(
    h_dE_rel_test_vs_E_rel_pred_pos, axis=3, only=9
)

In [ ]:
As_containment[9], bs_containment[9]

In [ ]:
watermark()

plt.axhline(res.params[0].value, color="r", label=r"nominal ($\pm1\;\sigma$)")
plt.axhline(res.params[0].value - res.params[0].error, linestyle="--", color="r")
plt.axhline(res.params[0].value + res.params[0].error, linestyle="--", color="r")
plt.errorbar(
    range(10),
    [A.value for A in As_x],
    yerr=[A.error for A in As_x],
    fmt="o",
    capsize=3,
    label=r"vary $x$",
)
plt.errorbar(
    range(10),
    [A.value for A in As_y],
    yerr=[A.error for A in As_y],
    fmt="o",
    capsize=3,
    label=r"vary $y$",
)
plt.errorbar(
    range(10),
    [A.value for A in As_z],
    yerr=[A.error for A in As_z],
    fmt="o",
    capsize=3,
    label=r"vary $z$",
)
plt.errorbar(
    range(10),
    [A.value for A in As_containment],
    yerr=[A.error for A in As_containment],
    fmt="o",
    capsize=3,
    label=r"vary containement",
)
plt.legend(ncols=2, loc=0, fontsize=10)
plt.xlabel("bin")
plt.ylabel(r"$A$")
plt.savefig(f"plots/As_{model_name}.pdf")
plt.savefig(f"plots/As_{model_name}.png")

In [ ]:
plt.axhline(res.params[1].value, color="r", label=r"nominal ($\pm1\;\sigma$)")
plt.axhline(res.params[1].value - res.params[1].error, linestyle="--", color="r")
plt.axhline(res.params[1].value + res.params[1].error, linestyle="--", color="r")

plt.errorbar(
    range(10),
    [b.value for b in bs_x],
    yerr=[b.error for b in bs_x],
    fmt="o",
    capsize=3,
    label=r"vary $x$",
)
plt.errorbar(
    range(10),
    [b.value for b in bs_y],
    yerr=[b.error for b in bs_y],
    fmt="o",
    capsize=3,
    label=r"vary $y$",
)
plt.errorbar(
    range(10),
    [b.value for b in bs_z],
    yerr=[b.error for b in bs_z],
    fmt="o",
    capsize=3,
    label=r"vary $z$",
)
plt.errorbar(
    range(10),
    [b.value for b in bs_containment],
    yerr=[b.error for b in bs_containment],
    fmt="o",
    capsize=3,
    label=r"vary containement",
)
plt.legend(ncols=2, loc=0, fontsize=10)
watermark()

plt.xlabel("bin")
plt.ylabel(r"$b$")
plt.savefig(f"plots/bs_{model_name}.pdf")
plt.savefig(f"plots/bs_{model_name}.png")

In [ ]:
import matplotlib.transforms as transforms

In [ ]:
def confidence_ellipse(mean_x, mean_y, cov, ax, n_std=1.0, facecolor="none", **kwargs):
    """
    Create a plot of the covariance confidence ellipse.

    Parameters
    ----------
    cov : array-like, shape (2, 2)
        Covariance

    ax : matplotlib.axes.Axes
        The Axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse(
        (0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs,
    )

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std

    transf = (
        transforms.Affine2D()
        .rotate_deg(45)
        .scale(scale_x, scale_y)
        .translate(mean_x, mean_y)
    )

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
plt.figure()
plt.errorbar(
    [A.value for A in As_x],
    [b.value for b in bs_x],
    xerr=[A.error for A in As_x],
    yerr=[b.error for b in bs_x],
    fmt="o",
    capsize=3,
    label=r"vary $x$",
)
for n, point in enumerate(zip([A.value for A in As_x], [b.value for b in bs_x])):
    plt.annotate(
        f"$x_{n}$",
        point,
        xytext=(5, 5),
        textcoords="offset pixels",
    )
plt.errorbar(
    [A.value for A in As_y],
    [b.value for b in bs_y],
    xerr=[A.error for A in As_y],
    yerr=[b.error for b in bs_y],
    fmt="o",
    capsize=3,
    label=r"vary $y$",
)
for n, point in enumerate(zip([A.value for A in As_y], [b.value for b in bs_y])):
    plt.annotate(
        f"$y_{n}$",
        point,
        xytext=(5, 5),
        textcoords="offset pixels",
    )
plt.errorbar(
    [A.value for A in As_z],
    [b.value for b in bs_z],
    xerr=[A.error for A in As_z],
    yerr=[b.error for b in bs_z],
    fmt="o",
    capsize=3,
    label=r"vary $z$",
)
for n, point in enumerate(zip([A.value for A in As_z], [b.value for b in bs_z])):
    plt.annotate(
        f"$z_{n}$",
        point,
        xytext=(5, 5),
        textcoords="offset pixels",
    )
plt.errorbar(
    [A.value for A in As_containment],
    [b.value for b in bs_containment],
    xerr=[A.error for A in As_containment],
    yerr=[b.error for b in bs_containment],
    fmt="o",
    capsize=3,
    label=r"vary containment",
)
for n, point in enumerate(
    zip([A.value for A in As_containment], [b.value for b in bs_containment])
):
    plt.annotate(
        f"$c_{n}$",
        point,
        xytext=(5, 5),
        textcoords="offset pixels",
    )
ax = plt.gca()
confidence_ellipse(
    res.params[0].value,
    res.params[1].value,
    res.covariance,
    ax,
    edgecolor="red",
    n_std=1,
    linewidth=2,
    facecolor="r",
    alpha=0.5,
)
confidence_ellipse(
    res.params[0].value,
    res.params[1].value,
    res.covariance,
    ax,
    edgecolor="red",
    n_std=2,
    linewidth=2,
    alpha=0.5,
)
confidence_ellipse(
    res.params[0].value,
    res.params[1].value,
    res.covariance,
    ax,
    edgecolor="red",
    n_std=3,
    linewidth=2,
    alpha=0.5,
)
plt.scatter(res.params[0].value, res.params[1].value, color="r", label=r"nominal")

plt.xlabel(r"$A$")
plt.ylabel(r"$b$")
plt.legend(ncols=2, loc=0, fontsize=10)
watermark()

plt.savefig(f"plots/As_and_bs_{model_name}.pdf")
plt.savefig(f"plots/As_and_bs_{model_name}.png")

In [ ]:
df

In [ ]:
df.visible.hist()